In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\| **Quickstart** \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Quickstart
==========

This section runs through the API for common tasks in machine learning.
Refer to the links in each section to dive deeper.

Working with data
-----------------

PyTorch has two [primitives to work with
data](https://pytorch.org/docs/stable/data.html):
`torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset`
stores the samples and their corresponding labels, and `DataLoader`
wraps an iterable around the `Dataset`.


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as
[TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and
[TorchAudio](https://pytorch.org/audio/stable/index.html), all of which
include datasets. For this tutorial, we will be using a TorchVision
dataset.

The `torchvision.datasets` module contains `Dataset` objects for many
real-world vision data like CIFAR, COCO ([full list
here](https://pytorch.org/vision/stable/datasets.html)). In this
tutorial, we use the FashionMNIST dataset. Every TorchVision `Dataset`
includes two arguments: `transform` and `target_transform` to modify the
samples and labels respectively.

torchvision自带有datasets组件，该组件中包含FashinMNIST数据集，root="data"用于指定数据集下载到本地的文件夹。FashionMNIST 的图像是单通道（灰度图），原始形状为 (H, W) = (28, 28)。
ToTensor() 会添加一个通道维度，调整为 PyTorch 期望的格式 (C, H, W) = (1, 28, 28)（符合卷积层等操作的输入要求）。

training_data的形态的理解：

检查单个样本

image, label = training_data[0]

print(type(image))   # <class 'torch.Tensor'>

print(image.shape)   # torch.Size([1, 28, 28]) (C, H, W)

print(label)         # int (e.g., 9)

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 10.8MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 169kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.31MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.8MB/s]


We pass the `Dataset` as an argument to `DataLoader`. This wraps an
iterable over our dataset, and supports automatic batching, sampling,
shuffling and multiprocess data loading. Here we define a batch size of
64, i.e. each element in the dataloader iterable will return a batch of
64 features and labels.

batch_size = 64表示每次处理64个样本（如图片）

DataLoader会自动将数据集分成多个批次，每次迭代返回一个包含64个样本及其标签的元组(X, y)。
train_dataloader = DataLoader(..., shuffle=True)  # 训练集通常需要打乱顺序。
DataLoader(..., num_workers=4)  # 使用4个子进程并行加载数据。
DataLoader(...,pin_memory=True if torch.cuda.is_available() else False)# 支持pin_memory，可加速GPU数据传输（当使用CUDA时）

X.shape输出中的维度含义：[N, C, H, W]分别代表：
N: batch size (64)
C: channels (如RGB图像的3通道)
H: height (图像高度)
W: width (图像宽度)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Read more about [loading data in PyTorch](data_tutorial.html).


------------------------------------------------------------------------


Creating Models
===============

To define a neural network in PyTorch, we create a class that inherits
from
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
We define the layers of the network in the `__init__` function and
specify how data will pass through the network in the `forward`
function. To accelerate operations in the neural network, we move it to
the
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"这条语句中torch.accelerator.current_accelerator().type if torch.accelerator.is_available()是一体的，即if成立使用torch.accelerator.current_accelerator().type赋值，不成立用字符串cpu赋值。

self.flatten = nn.Flatten() 是创建一个Flatten层实例，会把任意维度的输入张量转换为二维张量：第0维度保留为batch大小（样本数量），其他所有维度被展平成一个维度。在MNIST数据集的情况下（假设这是你正在使用的），输入通常是形状为 [batch_size, 1, 28, 28] 的四维张量（批量大小×通道数×高度×宽度），当数据通过这个Flatten层时：

原始形状：[batch_size, 1, 28, 28]

Flatten后形状：[batch_size, 1\*28\*28] = [batch_size, 784]

这样做的目的是将二维图像数据转换为一维向量，以便后续的全连接层（nn.Linear）能够处理。self.flatten = nn.Flatten()将这个层作为模型的一个永久组件保存下来，可以在forward方法中重复使用

self.linear_relu_stack = nn.Sequential(...)这段代码定义了一个名为 linear_relu_stack 的神经网络模块，它使用了 PyTorch 的 nn.Sequential 容器将多个线性层（全连接层）和 ReLU 激活函数按顺序堆叠在一起。下面是对代码的逐步解释：

1.nn.Sequential这是一个顺序容器，模块会按照它们在构造函数中传递的顺序被依次添加到其中。
输入数据会依次通过这些模块，前一个模块的输出是后一个模块的输入。

2.网络结构逐层说明

第一层：nn.Linear($28\times28$, 512).这是一个全连接层（线性变换），输入特征维度为 $28\times28$（假设输入是展平后的 MNIST 图像，尺寸为 $28 \times 28 = 784$），输出维度为 512。数学形式：$y = Wx + b$，其中 $W$ 是权重矩阵，$b$ 是偏置。

第二层：nn.ReLU(). ReLU（Rectified Linear Unit）激活函数，定义为 $\text{ReLU}(x) = \max(0, x)$。引入非线性性，增强网络的表达能力。

第三层：nn.Linear(512, 512). 又一个全连接层，输入和输出维度均为 512（隐藏层）。

第四层：nn.ReLU(). 再次使用 ReLU 激活函数。

第五层：nn.Linear(512, 10). 最后的全连接层，输出维度为 10（假设是 MNIST 的 10 分类任务）。

参数数量：该网络共有约 $(784 \times512 +512) + (512 \times512 +512) + (512 \times10 +10) =669,706$个可训练参数。

In [5]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Read more about [building neural networks in
PyTorch](buildmodel_tutorial.html).


------------------------------------------------------------------------


Optimizing the Model Parameters
===============================

To train a model, we need a [loss
function](https://pytorch.org/docs/stable/nn.html#loss-functions) and an
[optimizer](https://pytorch.org/docs/stable/optim.html).


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training
dataset (fed to it in batches), and backpropagates the prediction error
to adjust the model\'s parameters.


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model\'s performance against the test dataset to
ensure it is learning.


In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*).
During each epoch, the model learns parameters to make better
predictions. We print the model\'s accuracy and loss at each epoch;
we\'d like to see the accuracy increase and the loss decrease with every
epoch.


In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.311366  [   64/60000]
loss: 2.292379  [ 6464/60000]
loss: 2.270364  [12864/60000]
loss: 2.259991  [19264/60000]
loss: 2.237917  [25664/60000]
loss: 2.214977  [32064/60000]
loss: 2.221948  [38464/60000]
loss: 2.192616  [44864/60000]
loss: 2.183903  [51264/60000]
loss: 2.143116  [57664/60000]
Test Error: 
 Accuracy: 47.1%, Avg loss: 2.143935 

Epoch 2
-------------------------------
loss: 2.161515  [   64/60000]
loss: 2.152451  [ 6464/60000]
loss: 2.089290  [12864/60000]
loss: 2.103713  [19264/60000]
loss: 2.053924  [25664/60000]
loss: 1.983912  [32064/60000]
loss: 2.019530  [38464/60000]
loss: 1.944025  [44864/60000]
loss: 1.936783  [51264/60000]
loss: 1.860970  [57664/60000]
Test Error: 
 Accuracy: 60.4%, Avg loss: 1.869253 

Epoch 3
-------------------------------
loss: 1.899861  [   64/60000]
loss: 1.878893  [ 6464/60000]
loss: 1.756708  [12864/60000]
loss: 1.798471  [19264/60000]
loss: 1.685683  [25664/60000]
loss: 1.628623  [32064/600

Read more about [Training your model](optimization_tutorial.html).


------------------------------------------------------------------------


Saving Models
=============

A common way to save a model is to serialize the internal state
dictionary (containing the model parameters).


In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
==============

The process for loading a model includes re-creating the model structure
and loading the state dictionary into it.


In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

This model can now be used to make predictions.


In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


Read more about [Saving & Loading your
model](saveloadrun_tutorial.html).
